## Setup

In [1]:
import csv
import pandas as pd
import numpy as np

train_df = pd.read_csv("training.csv", delimiter=",", header=0)
test_df = pd.read_csv("testing.csv", delimiter=",", header=0)
confidence_df = pd.read_csv("annotation_confidence.csv", delimiter=",", header=0)
additional_df = pd.read_csv("additional_training.csv", delimiter=",", header=0)
proportions_df = pd.read_csv("test_proportions.csv", delimiter=",", header=None, dtype="unicode")

all_df = pd.concat([train_df, additional_df])

unconfident_list = confidence_df[confidence_df.confidence != 1]['ID'].tolist()
confident_list = confidence_df[confidence_df.confidence == 1]['ID'].tolist()

## Confident Training Average

 - Training Data
 - Confident Training Data
 - Confident 0 Training Data
 - Confident 1 Trainng Data
 - Averages for above 2

In [2]:
confident_train_df = train_df[train_df['ID'].isin(confident_list)]
confident_0_train_df = confident_train_df[confident_train_df.prediction == 0]
confident_1_train_df = confident_train_df[confident_train_df.prediction == 1]
confident_0_avg = pd.DataFrame(confident_0_train_df.mean(axis=0)[1:-1]).T
confident_1_avg = pd.DataFrame(confident_1_train_df.mean(axis=0)[1:-1]).T

## Unconfident Training and Additional Average

 - Unconfident All Data
 - Unconfident 0 All Data
 - Unconfident 1 All Data
 - Averages for above 2

In [3]:
unconfident_all_df =  all_df[all_df['ID'].isin(unconfident_list)]
unconfident_0_all_df = unconfident_all_df[unconfident_all_df.prediction == 0]
unconfident_1_all_df = unconfident_all_df[unconfident_all_df.prediction == 1]
unconfident_0_avg = pd.DataFrame(unconfident_0_all_df.mean(axis=0)[1:-1]).T
unconfident_1_avg = pd.DataFrame(unconfident_1_all_df.mean(axis=0)[1:-1]).T

## Fill Additional With Confident Averages

 - Additional Data
 - Confident Additional Data
 - Confident 0 Additional Data
 - Confident 1 Additional Data
 - FillNa above 2 with Confident Averages
 - Update Additional with New values

In [4]:
confident_additional_df = additional_df[additional_df['ID'].isin(confident_list)].copy()
confident_0_additional_df = confident_additional_df[confident_additional_df.prediction == 0]
confident_1_additional_df = confident_additional_df[confident_additional_df.prediction == 1]
confident_0_additional_df = confident_0_additional_df.fillna(value=confident_0_avg.iloc[0])
confident_1_additional_df = confident_1_additional_df.fillna(value=confident_1_avg.iloc[0])

In [5]:
confident_additional_df.update(confident_0_additional_df)
confident_additional_df.update(confident_1_additional_df)
additional_df.update(confident_additional_df)

## Fill Additional With Unconfident Averages

 - Unconfident AdditionalData
 - Unconfident 0 Additional Data
 - Unconfident 1 Additional Data
 - FillNa above 2 with Unconfident Averages
 - Update Additional with New Values


In [6]:
unconfident_additional_df = additional_df[additional_df['ID'].isin(unconfident_list)].copy()
unconfident_0_additional_df = unconfident_additional_df[unconfident_additional_df.prediction == 0]
unconfident_1_additional_df = unconfident_additional_df[unconfident_additional_df.prediction == 1]
unconfident_0_additional_df = unconfident_0_additional_df.fillna(value=unconfident_0_avg.iloc[0])
unconfident_1_additional_df = unconfident_1_additional_df.fillna(value=unconfident_1_avg.iloc[0])

In [7]:
unconfident_additional_df.update(unconfident_0_additional_df)
unconfident_additional_df.update(unconfident_1_additional_df)
additional_df.update(unconfident_additional_df)

### Update All with New Additional Values

In [8]:
all_df.update(additional_df)

# Multi-Layered Perceptron

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random

#Standardization
sc=StandardScaler()

In [17]:
all_input_data = all_df.drop(['ID', 'prediction'], axis=1).copy()
all_output_data = all_df['prediction'].copy()
all_input_data_std = pd.DataFrame(sc.fit_transform(all_input_data))

all_input_data_train, all_input_data_test, all_output_data_train, all_output_data_test = train_test_split(all_input_data,
                                                                                                          all_output_data,
                                                                                                         test_size=0.3,
                                                                                                         random_state=2018)
all_input_data_train_std = pd.DataFrame(sc.fit_transform(all_input_data_train))
all_input_data_test_std = pd.DataFrame(sc.transform(all_input_data_test))
display(all_input_data_train_std[:5])
display(all_input_data_test_std[:5])

# Official Testing Data
testing_input_data = test_df.drop(['ID'], axis=1).copy()
testing_input_data_std = pd.DataFrame(sc.transform(testing_input_data))

#np.random.seed(2018) No Idea
#random.seed(2018)

,0,1,2,3,4,5,6,7,8,9,...,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607
0,-0.486645,-0.392660,-0.359750,-0.485127,-0.459488,-0.540476,-0.666244,-0.661199,-0.577916,0.747233,...,0.293899,-0.166834,0.369458,0.386308,-0.492837,-0.739001,0.303951,-0.242137,-1.244617,-1.064447
1,-0.486645,0.127280,-0.359750,-0.485127,-0.459488,0.603111,-0.666244,2.419194,-0.577916,0.052351,...,0.514554,0.077314,-0.249538,0.032557,0.315849,-0.811516,0.430182,-0.378704,0.096350,0.864767
2,-0.326229,0.803328,0.097264,1.015179,-0.459488,1.639548,-0.666244,-0.109270,-0.577916,-0.759306,...,0.040617,0.688078,1.504918,2.603867,1.758018,1.879006,1.037315,0.072729,0.407271,0.743069
3,-0.486645,-0.535811,-0.359750,6.254173,-0.459488,-0.540476,0.258153,0.155257,1.267018,1.710574,...,-1.199802,-0.805845,-0.227173,-1.343459,-1.216297,-0.762978,-0.197561,-1.521580,-0.141262,-0.425687
4,-0.486645,-0.109763,-0.129671,-0.018942,3.574124,-0.540476,-0.352161,-0.661199,0.134694,-0.759306,...,-0.964901,0.167138,-0.546753,-1.089281,-0.081190,0.004939,-1.095942,-1.816843,-1.982592,-0.506644


,0,1,2,3,4,5,6,7,8,9,...,4598,4599,4600,4601,4602,4603,4604,4605,4606,4607
0,-0.486645,-0.535811,-0.179993,-0.485127,-0.459488,-0.191879,0.891121,0.179527,0.134694,-0.759306,...,-0.054173,1.172633,-0.094939,0.435587,-0.081190,-0.123026,-0.398794,0.067198,0.474355,1.835503
1,1.278980,-0.535811,-0.262127,-0.485127,-0.459488,0.080470,-0.608635,4.425808,1.291836,-0.144921,...,1.871160,-0.493207,0.095716,-0.623637,-0.612415,-1.023784,0.981665,1.165998,0.643927,-0.665792
2,-0.486645,-0.535811,-0.359750,-0.485127,-0.459488,-0.540476,-0.666244,-0.661199,0.134694,-0.075975,...,-0.054173,-0.459087,-0.094939,-1.752985,-1.035105,-0.123026,0.128299,-0.238897,-0.281905,-1.408399
3,-0.486645,-0.535811,-0.359750,-0.485127,-0.459488,-0.540476,-0.666244,-0.078615,-0.577916,-0.075975,...,-0.502868,2.946900,-0.736962,-1.315458,-1.025054,-0.123026,-0.797343,-1.328894,-1.366281,0.105953
4,-0.486645,-0.535811,-0.359750,3.150208,-0.459488,0.887672,-0.666244,-0.344442,0.393186,2.401618,...,-0.399309,-0.310842,-0.641103,-0.101099,-0.670518,-0.123026,-0.131376,-0.098457,1.079729,1.002536


## Create MLP Model

In [18]:
#create model
model = Sequential()
model.add(Dense(36, input_dim=4608, activation='sigmoid'))
model.add(Dense(18, activation='sigmoid'))
model.add(Dense(9, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.summary()

In [19]:
#train model
model.fit(all_input_data_train_std, all_output_data_train, epochs=100, batch_size=72, verbose=0)

#evaluate training
scores = model.evaluate(all_input_data_train_std, all_output_data_train)
#print("\n%s: %.5f%%" % (model.metrics_names[0], scores[0]*100))
#print("\n%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))

3192/3192 [==============================] - 1s 364us/step


## Make Predictions

In [23]:
#make predictions on test data
predictions = model.predict(all_input_data_test_std)
rounded = [round(x[0]) for x in predictions]

accuracy = accuracy_score(all_output_data_test, rounded)
print('Accuracy: ', accuracy)

val1 = (rounded.count(1) / 5040) *100
val0 = (rounded.count(0) / 5040) *100
print('Ones:' , val1)
print('Zeros:', val0)
#display(proportions_df)

Accuracy:  0.863304093567
Ones: 15.892857142857142
Zeros: 11.25


In [ ]:
#write results to file
#results_df.to_csv('submission5.csv', index=False)